In [1]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.insert(0, "..")

# K-folding

* Let's try to k-fold our data.
* Testing our model performance on different train and valid sets increases the likelihood that our model
* (and getting good results with low variance) will generalize better to unseen data 
* Since our targets are sequences, getting getting the "comparable" distribution will take a little bit of creativity, as normal stratified k folding won't work and we kind of have to establish what "distribution" we have in mind
* ^ But let's just ignore that and do random Kfolding and pray to Lord Andrew that this will work out :D

In [2]:
from pathlib import Path
from collections import defaultdict
import joblib
from core.utils import split_into_k_folds 
from core.config import config

In [3]:
split_into_k_folds("../data/proc")

# Checking label ratios for our folds 

* We can check many things for validating the "goodnes-of-split", but let's limit ourselves by calculating ratios
* of our tags

In [4]:
folds_dir = Path("../data/k-fold"); assert folds_dir.exists()
 
tag_dists = {} 
for i in range(1, 11):
    fold_k_dir = folds_dir / f"fold{i}"; assert fold_k_dir.exists()
    
    train_k = joblib.load(fold_k_dir / "train.bin")
    val_k = joblib.load(fold_k_dir / "val.bin")
    print(f"TAIN VAL LENGTHS OF FOLD{i}: {len(train_k[0])} {len(val_k[0])}")
    
    train_k_tags = defaultdict(lambda : 0)
    for tag_seq in train_k[1]:
        for tag in tag_seq:
            train_k_tags[tag] +=1
     
    val_k_tags = defaultdict(lambda : 0)
    for tag_seq in val_k[1]:
        for tag in tag_seq:
            val_k_tags[tag] +=1
    
    tag_dists[i] = {"train": train_k_tags, "val": val_k_tags}

TAIN VAL LENGTHS OF FOLD1: 6541 727
TAIN VAL LENGTHS OF FOLD2: 6541 727
TAIN VAL LENGTHS OF FOLD3: 6541 727
TAIN VAL LENGTHS OF FOLD4: 6541 727
TAIN VAL LENGTHS OF FOLD5: 6541 727
TAIN VAL LENGTHS OF FOLD6: 6541 727
TAIN VAL LENGTHS OF FOLD7: 6541 727
TAIN VAL LENGTHS OF FOLD8: 6541 727
TAIN VAL LENGTHS OF FOLD9: 6542 726
TAIN VAL LENGTHS OF FOLD10: 6542 726


In [5]:
for i in range(1, 11):
    t = tag_dists[i]["train"] 
    v = tag_dists[i]["val"]
    print(f"FOLD{i} | {t[0], t[1], t[2]} | {v[0], v[1], v[2]}") 

FOLD1 | (6006, 7482, 152070) | (698, 809, 16468)
FOLD2 | (6039, 7426, 151823) | (665, 865, 16715)
FOLD3 | (5944, 7324, 151616) | (760, 967, 16922)
FOLD4 | (6010, 7493, 150889) | (694, 798, 17649)
FOLD5 | (6022, 7378, 151646) | (682, 913, 16892)
FOLD6 | (6062, 7491, 151786) | (642, 800, 16752)
FOLD7 | (6078, 7568, 151505) | (626, 723, 17033)
FOLD8 | (6094, 7536, 151321) | (610, 755, 17217)
FOLD9 | (6042, 7485, 152172) | (662, 806, 16366)
FOLD10 | (6039, 7436, 152014) | (665, 855, 16524)


* IDK, seems pretty good, a bit TOO good
* Would probably need a little bit additional verification to check if folds are not too similar/dupplicate, but for now let's do a simple check

In [6]:
train_5 = set(" ".join(i) for i in joblib.load(folds_dir/"fold2"/"train.bin")[0])
train_9 = set(" ".join(i) for i in joblib.load(folds_dir/"fold9"/"train.bin")[0])
len(train_5 & train_9)

5223

* Seems about right, since our train set is like 6k samples and valid is 800 it  woul make sense that 5.2k match